# Query Transformations
Query transformations are a set of approaches focused on re-writing and / or modifying questions for retrieval.

![Alt text](query_transform.png)


In [6]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [7]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_PROJECT'] = 'advanced-rag'
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")

# Part 1: Multi Query

![Alt text](multi_query.png)




Docs:

https://python.langchain.com/docs/modules/data_connection/retrievers/MultiQueryRetriever


### Index

In [8]:
#### INDEXING ####

import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

# Index
model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)
vectorstore = FAISS.from_documents(documents=splits, 
                                    embedding=hf_embeddings)

retriever = vectorstore.as_retriever()

### Multi Query Prompt

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# Multi Query: One Question, Multiple Perspectives.

template="""
You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""


prompt_perspectives = ChatPromptTemplate.from_template(template)




generate_queries=(prompt_perspectives | ChatGroq(model="openai/gpt-oss-120b", temperature=0)|StrOutputParser()| (lambda x: x.split("\n")))

In [10]:
generate_queries.invoke("What is Task Decomposition")

['What does the term “task decomposition” mean?  ',
 'How is task decomposition defined in the context of problem solving?  ',
 'Can you explain the concept of breaking a task into sub‑tasks?  ',
 'What are the principles and steps involved in task decomposition?  ',
 'In which fields is task decomposition used and why is it important?']

In [11]:
# Load Unique documents only to avoid duplicates to be retrieved

from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]

    # Get unique documents
    unique_docs = list(set(flattened_docs))

    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve
question = "What is task decomposition for LLM agents?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

C:\Users\HAROON TRADERS\AppData\Local\Temp\ipykernel_9560\3066567925.py:14: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  return [loads(doc) for doc in unique_docs]


9

In [12]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:{context}

Question: {question}

"""


prompt = ChatPromptTemplate.from_template(template)

llm= ChatGroq(model="openai/gpt-oss-120b", temperature=0)

rag_chain= ({"context":retrieval_chain,"question":itemgetter("question")} | prompt | llm  | StrOutputParser())


rag_chain.invoke({"question":question})


'**Task decomposition** is the process by which an LLM‑based autonomous agent breaks a large, complex request into a series of smaller, manageable sub‑tasks (or sub‑goals) that can be solved step‑by‑step.  \n\nKey points:\n\n| Aspect | Description |\n|--------|-------------|\n| **Purpose** | Turns a “big” problem into a sequence of simpler actions, making planning, execution, and error‑recovery easier for the agent. |\n| **Techniques** | • **Chain‑of‑Thought (CoT)** prompting – ask the model to “think step‑by‑step” so it explicitly lists intermediate steps.<br>• **Tree‑of‑Thought (ToT)** – generate multiple possible thoughts at each step and search (BFS/DFS) among them, allowing exploration of alternative solution paths.<br>• **Prompt‑driven instructions** – e.g., “What are the subgoals for achieving X?” or “List the steps for Y.” |\n| **Implementation** | The LLM receives a prompt that requests a decomposition (often in a numbered list). The output can be used directly as a task queue

# Part 2: RAG-Fusion

![Alt text](RAG_Fusion.png)


### Prompt

In [13]:
from langchain.prompts import ChatPromptTemplate

# RAG-Fusion: Related
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""

prompt_rag_fusion = ChatPromptTemplate.from_template(template)

In [14]:
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq

generate_queries = (
    prompt_rag_fusion 
    | ChatGroq(model="openai/gpt-oss-20b", temperature=0)
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [15]:
generate_queries.invoke("What is Task Decomposition for LLM agents?")

['1. “Task decomposition in large language model agents: definition and key concepts”  ',
 '2. “How do LLM agents perform task decomposition? Step‑by‑step examples”  ',
 '3. “Benefits of task decomposition for autonomous LLM agents”  ',
 '4. “Implementing task decomposition in LLM‑based agents: code snippets and best practices”']

# RRF Explanation

## Question Rankings
- **Question A**  
  1. Doc1  
  2. Doc4  
  3. Doc3  
  4. Doc2  

- **Question B**  
  1. Doc3  
  2. Doc1  
  3. Doc2  
  4. Doc4  

- **Question C**  
  1. Doc4  
  2. Doc3  
  3. Doc1  
  4. Doc2  

---

## Rank Positions
- **Doc1**  
  - Question A rank: **1**  
  - Question B rank: **2**  
  - Question C rank: **3**  

- **Doc2**  
  - Question A rank: **4**  
  - Question B rank: **3**  
  - Question C rank: **4**  

- **Doc3**  
  - Question A rank: **3**  
  - Question B rank: **1**  
  - Question C rank: **2**  

- **Doc4**  
  - Question A rank: **2**  
  - Question B rank: **4**  
  - Question C rank: **1**  

---

## Reciprocal Rank Fusion Calculation

![Alt text](RRF.png)


---

## Final Ranking
1. **Doc1** ≈ 0.0487  
2. **Doc3** ≈ 0.0487  
3. **Doc4** ≈ 0.0484  
4. **Doc2** ≈ 0.0469  

---

✅ **Conclusion**: Both **Doc1** and **Doc3** achieve the highest RRF scores, making them the most relevant across the question rankings.


In [16]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents 
        and an optional parameter k used in the RRF formula """
    
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question": question})
len(docs)

9

In [17]:
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'**Task decomposition** is the process by which an LLM‑driven autonomous agent breaks a large, complex request into a series of smaller, manageable sub‑tasks (or sub‑goals) that can be solved step‑by‑step.  \n\nKey points from the Lilian\u202fWeng article:\n\n| Aspect | Description |\n|--------|-------------|\n| **Purpose** | Enables the agent to handle complicated problems that would be too difficult to solve in a single pass. By creating intermediate milestones, the agent can plan ahead, track progress, and recover from errors. |\n| **How it’s done** | • **Prompt‑based prompting** – simple instructions such as “Steps for\u202fXYZ.\u202f1.” or “What are the subgoals for achieving\u202fXYZ?” can coax the LLM to list sub‑tasks. <br>• **Chain‑of‑Thought (CoT)** – the model is asked to “think step‑by‑step,” which naturally yields a linear decomposition of the problem. <br>• **Tree‑of‑Thought (ToT)** – extends CoT by generating multiple candidate thoughts at each step and organizing them i

# Part 3:Decomposition

In [18]:
from langchain.prompts import ChatPromptTemplate

# Decomposition
template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
Generate multiple search queries related to: {question} \n
Output (3 queries):"""

prompt_decomposition = ChatPromptTemplate.from_template(template)

In [19]:
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser

# LLM
llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0)

# Chain
generate_queries_decomposition = ( prompt_decomposition | llm | StrOutputParser() | (lambda x: x.split("\n")))

# Run
question = "What are the main components of an LLM-powered autonomous agent system?"

questions = generate_queries_decomposition.invoke({"question":question})

In [20]:
questions


['To break down the question "What are the main components of an LLM-powered autonomous agent system?" into manageable sub-questions, we can consider the following search queries:',
 '',
 '1. **What are the key architectural elements of an autonomous agent system?**',
 '   - This query focuses on understanding the overall structure and fundamental parts that make up an autonomous agent system, which can include perception, decision-making, and action components.',
 '',
 "2. **How does a Large Language Model (LLM) integrate with an autonomous agent's decision-making process?**",
 '   - This search query delves into the specifics of how LLMs contribute to the autonomous decision-making process, potentially exploring aspects like natural language processing, knowledge graph integration, and reasoning mechanisms.',
 '',
 '3. **What role do sensing and perception components play in an LLM-powered autonomous agent, and how do they interact with the LLM?**',
 '   - This query investigates the

Recursive 

![Alt text](Decomposition.png)

In [21]:
# Prompt
template = """Here is the question you need to answer:

\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question: 

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""

decomposition_prompt = ChatPromptTemplate.from_template(template)

In [28]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

def format_qa_pair(question, answer):
    """Format Q and A pair"""
    
    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
    return formatted_string.strip()

# llm
llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0, max_tokens=512)


q_a_pairs = ""
for q in questions:
    
    rag_chain = (
    {"context": itemgetter("question") | retriever, 
     "question": itemgetter("question"),
     "q_a_pairs": itemgetter("q_a_pairs")} 
    | decomposition_prompt
    | llm
    | StrOutputParser())

    answer = rag_chain.invoke({"question":q,"q_a_pairs":q_a_pairs})
    q_a_pair = format_qa_pair(q,answer)
    q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jrz0501dearravrx7tr28sdv` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 100137, Requested 1307. Please try again in 20m48.383s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [ ]:
answer

'## 3.\u202fLLM‑Based Autonomous Agent System Building Blocks  \n\nBelow is a concise, modular view of the **building blocks** that compose a modern autonomous agent whose core controller is a Large Language Model (LLM).  \nThe list is organized as a *layered architecture* (perception → cognition → action → learning → governance) and is directly derived from the survey by Lilian\u202fWeng (2023) and the component‑level answer in the background material.\n\n| # | Building Block | Core Responsibility | Typical Implementation / Example | Why It Matters |\n|---|---------------|-------------------|--------------------------------|---------------|\n| **1** | **LLM Core (Brain)** | Generates plans, decisions, natural‑language responses, and code. | GPT‑4, Claude‑3, Llama‑2‑70B, etc. | The “brain” that drives all other blocks. |\n| **2** | **Perception / Input Processing** | Converts raw sensory data (text, images, sensor streams) into a format the LLM can consume. | OCR, CLIP, image captionin



![Alt text](Decomposition_2.png)

In [ ]:
# Answer each sub-question individually 

from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq

# RAG prompt
prompt_rag = hub.pull("rlm/rag-prompt")

def retrieve_and_rag(question,prompt_rag,sub_question_generator_chain):
    """RAG on each sub-question"""
    
    # Use our decomposition / 
    sub_questions = sub_question_generator_chain.invoke({"question":question})
    
    # Initialize a list to hold RAG chain results
    rag_results = []
    
    for sub_question in sub_questions:
        
        # Retrieve documents for each sub-question
        retrieved_docs = retriever.get_relevant_documents(sub_question)
        
        # Use retrieved documents and sub-question in RAG chain
        answer = (prompt_rag | llm | StrOutputParser()).invoke({"context": retrieved_docs, 
                                                                "question": sub_question})
        rag_results.append(answer)
    
    return rag_results,sub_questions

# Wrap the retrieval and RAG process in a RunnableLambda for integration into a chain
answers, questions = retrieve_and_rag(question, prompt_rag, generate_queries_decomposition)

In [ ]:
def format_qa_pairs(questions, answers):
    """Format Q and A pairs"""
    
    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"
    return formatted_string.strip()

context = format_qa_pairs(questions, answers)

# Prompt
template = """Here is a set of Q+A pairs:

{context}

Use these to synthesize an answer to the question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context":context,"question":question})

'**Main Components of an LLM‑Powered Autonomous Agent System**\n\n| # | Component | Core Responsibility | Typical Sub‑modules / Techniques |\n|---|----------|-------------------|--------------------------------|\n| **1. LLM Core** | Acts as the “brain” – generates text, decisions, and high‑level plans. | Prompt‑engineering, chain‑of‑thought, Tree‑of‑Thoughts, ReAct, etc. |\n| **2. Planning & Reflection** | Breaks tasks into sub‑goals, refines actions, and self‑criticises. | Sub‑goal decomposition, self‑reflection loops, policy‑updates. |\n| **3. Memory** | Stores short‑term context and long‑term knowledge for retrieval and updating. | Episodic memory, vector‑store, knowledge graph, retrieval‑augmented generation. |\n| **4. Tool / Interface Layer** | Enables the agent to interact with the outside world (APIs, web, code, hardware). | Tool routers (MRKL, Toolformer), API calls, web‑browser, code‑execution, hardware control. |\n\nThese four pillars—**LLM core, planning & reflection, memory

# Part 4:Step Back

![image.png](step_back.png)

In [24]:
# Few Shot Examples
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Jan Sindel’s was born in what country?",
        "output": "what is Jan Sindel’s personal history?",
    },
]
# We now transform these to example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        # Few shot examples
        few_shot_prompt,
        # New question
        ("user", "{question}"),
    ]
)

In [26]:
generate_queries_step_back = prompt | ChatGroq(model="llama-3.3-70b-versatile", temperature=0) | StrOutputParser()
question = "What is task decomposition for LLM agents?"
generate_queries_step_back.invoke({"question": question})

'How do complex tasks get broken down into simpler ones?'

In [ ]:
generate_queries_step_back = prompt | ChatGroq(model="llama-3.3-70b-versatile", temperature=0) | StrOutputParser()
question = "What is task decomposition for LLM agents?"
generate_queries_step_back.invoke({"question": question})

ValidationError: 1 validation error for ChatGroq
model
  Field required [type=missing, input_value={'temperature': 0, 'model_kwargs': {}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing